In [29]:
from scipy.io import arff
import pandas as pd
import numpy as np
import os
import operator
from classifier import classifier

In [36]:
class knn(classifier):
    def __init__(self, train_X, train_Y, test_X, test_Y):
        self.train_X = train_X
        self.train_Y = train_Y
        self.test_X = test_X
        self.test_Y = test_Y  
        self.hypothesis = None
        self.kn = 3
    
    def fit(self):
        fit(self.kn)
    
    def fit(self, kn):
        self.hypothesis = []
        self.kn = kn
        l_test = len(self.test_X)
        for i in range(l_test):
            kNears = self.getKNearest(self.test_X[i])
            result = self.getLabel(kNears)
            self.hypothesis.append(result)        
        return self.hypothesis

    def predict(self, pred_X):
        l_test = len(pred_X)
        predict_Y = []
        for i in range(l_test):
            kNears = self.getKNearest(pred_X[i])
            result = self.getLabel(kNears)
            predict_Y.append(result)        
        return predict_Y      
    
    def getAccuracy(self):
        correct = 0
        length = len(self.test_X)
        for i in range(length):
            if self.test_Y[i] == self.hypothesis[i]:
                correct += 1
        return (correct/float(length))
    
    def getDist(self, x1, x2):
        a = np.power((x1 - x2), 2)
        s = np.sum(a)
        return np.sqrt(s)
    
    def getKNearest(self, test_x):
        dists = []
        for i in range(len(self.train_X)):
            dist = self.getDist(test_x, self.train_X[i])
            dists.append((self.train_Y[i], dist))
        dists.sort(key=operator.itemgetter(1))
        neighbors = []
        for i in range(self.kn):
            neighbors.append(dists[i][0])
        return neighbors

    def getLabel(self, neighbors):
        labels = {}
        for i in range(len(neighbors)):
            label = neighbors[i]
            if label in labels:
                labels[label] += 1
            else:
                labels[label] = 1
        sortedLabels = sorted(labels.items(), key=operator.itemgetter(1), reverse=True)
        return sortedLabels[0][0]

In [37]:
# Function for converting arff list to csv list
def toCsv(content):
    data = False
    header = ""
    newContent = []
    for line in content:
        if not data:
            if "@attribute" in line:
                attri = line.split()
                columnName = attri[attri.index("@attribute")+1]
                header = header + columnName + ","
            elif "@data" in line:
                data = True
                header = header[:-1]
                header += '\n'
                newContent.append(header)
        else:
            newContent.append(line)
    return newContent

file = 'PhishingData.arff'
newFile = ''
with open(file , "r") as inFile:
    content = inFile.readlines()
    name,ext = os.path.splitext(inFile.name)
    new = toCsv(content)
    newFile = name + ".csv"
    with open(newFile, "w") as outFile:
        outFile.writelines(new)
        
df = pd.read_csv(newFile)

X = df.iloc[:,:-1]
Y = df.iloc[:,-1]
l = round(len(X)*0.8)
train_X = np.array(X[:l])
train_Y = np.array(Y[:l])
test_X = np.array(X[l:])
test_Y = np.array(Y[l:])

oop = knn(train_X, train_Y, test_X, test_Y)
for i in range(2,13):
    a = oop.fit(i)
    b = oop.getAccuracy()
    print(i, ":", b) 

2 : 0.8745387453874539
3 : 0.8856088560885609
4 : 0.8929889298892989
5 : 0.8892988929889298
6 : 0.8892988929889298
7 : 0.8929889298892989
8 : 0.8892988929889298
9 : 0.8966789667896679
10 : 0.8966789667896679
11 : 0.8929889298892989
12 : 0.8892988929889298
